# Geodesic Regression on Parameterized Hippocampus data!

## Step 1: get brain data to have same parameterization

we will get each brain mesh to have the same parameterization by: 
- choosing the first day to be the "template mesh".
- drawing a geodesic between the "template mesh" and all the other meshes using H2_SurfaceMatch
- H2_SurfaceMatch will make the parameterization of the target match the parameterization of the source.

In [5]:
import glob
import itertools
import multiprocessing
import os
import subprocess
import sys
import time
import warnings

import numpy as np
import torch
import trimesh

This will be done in main.py. You can "activate" the reparameterization code by going into default_config and making 

In [6]:
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()

parameterized_meshes_dir = os.path.join(
    os.getcwd(), "my28brains", "results", "meshes_parameterized"
)
h2_dir = os.path.join(work_dir, "H2_SurfaceMatch")
                      
sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
sys.path.append(h2_dir)

import H2_SurfaceMatch.H2_match  # noqa: E402
import H2_SurfaceMatch.utils.input_output  # noqa: E402
import H2_SurfaceMatch.utils.utils  # noqa: E402

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 1.1 Load Data

In [9]:
hemisphere = 'left'
structure_id = -1

string_base = os.path.join(
    parameterized_meshes_dir,
    f"{hemisphere}_structure_{structure_id}**.ply",
)
paths = sorted(glob.glob(string_base))
print(string_base)
print(paths)

/home/adele/code/my28brains/my28brains/results/meshes_parameterized/left_structure_-1**.ply
[]


In [ ]:
[
    vertices_source,
    faces_source,
    FunS,
] = H2_SurfaceMatch.utils.input_output.loadData(start_path)

# Step 2: Take the re-parameterized mesh sequence, and perform geodesic regression on it.